In [ ]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import ipysheet
import ipywidgets as widgets

In [ ]:
separateur = ";"
PAIRING_FILE = "~/Workspace/pairings/pairings.csv"

In [ ]:
df = pd.read_csv(PAIRING_FILE, sep=separateur, index_col=0)

In [ ]:
df

In [ ]:
sheet = ipysheet.from_dataframe(df)
widgets.VBox([sheet])
sheet

In [ ]:
def has_duplicates(listOfElements):
    if len(listOfElements) == len(set(listOfElements)):
        return False
    else:
        return True

In [ ]:
def sum_score_of_pairing(pairing):
    score = 0
    for (x, y) in pairing:
        score += df[x][y]
    return score

In [ ]:
def launch_pairing(instance):
    df = ipysheet.to_dataframe(sheet)
    team1=df.columns.values.tolist()
    team2=df.index.values.tolist()
    list_j1 = list(itertools.product([team1[0]], team2))
    list_j2 = list(itertools.product([team1[1]], team2))
    list_j3 = list(itertools.product([team1[2]], team2))
    list_j4 = list(itertools.product([team1[3]], team2))
    list_j5 = list(itertools.product([team1[4]], team2))
    list_j6 = list(itertools.product([team1[5]], team2))
    total = list(itertools.product(list_j1, list_j2, list_j3, list_j4, list_j5, list_j6))
    pairings = filter(lambda pairing: not has_duplicates([x[1] for x in pairing]), total)
    list_pairings = list(pairings)
    df_pairings = pd.DataFrame(pd.Series(list_pairings), columns=["Pairings"])
    df_pairings["Estimation"] = df_pairings["Pairings"].apply(lambda x: sum_score_of_pairing(x))
    df_hist = df_pairings.Estimation.value_counts().to_frame('count').reset_index() #.rename(columns={"index":"Estimation en VP", "count":"test"})
    df_hist["Estimation en VP"] = df_hist["index"]
    df_hist["Nombre de pairings"] = df_hist["count"]
    df_hist = df_hist[["Estimation en VP", "Nombre de pairings"]]
    display(df_hist.sort_values(by="Estimation en VP"))
    plt.hist(df_pairings.Estimation)
    plt.title("Répartition des estimations sur les 720 pairings")
    plt.xlabel("Estimation en VP")
    plt.ylabel("Nombre de pairings")
    plt.grid()

In [ ]:
def print_toto(instance):
    display("toto")

In [1]:
button = widgets.Button(
    description='Lancer le pairing',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Lancer le pairing',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(print_toto)
button

NameError: name 'widgets' is not defined